# 1 Preliminaries
## 1.1 Import
Import required libraries.

In [206]:
import numpy as np
from keras import backend as K
from keras import callbacks, models, layers, ops
import pandas as pd
import chess.pgn

## 1.2 Load Data
Load the pgn file and create DataFrame. It should have three columns:
- Color
- Board
- Move

In [207]:
def fen_to_board(fen):
    """Function written by ChatGPT"""
    piece_to_char = {
        'p': 'p', 'r': 'r', 'n': 'n', 'b': 'b', 'q': 'q', 'k': 'k',
        'P': 'P', 'R': 'R', 'N': 'N', 'B': 'B', 'Q': 'Q', 'K': 'K',
        '1': '.', '2': '.', '3': '.', '4': '.', '5': '.', '6': '.', '7': '.', '8': '.'
    }
    board = []
    for row in fen.split()[0].split('/'):
        board_row = []
        for char in row:
            if char.isdigit():
                board_row.extend(['.'] * int(char))
            else:
                board_row.append(piece_to_char[char])
        board.append(board_row)
    return np.array(board)

data = []
with open('data/master_games.pgn', 'r') as pgn:
    while True:
        game = chess.pgn.read_game(pgn)
        
        if game is None:
            break
            
        board = game.board()
        
        color = "white" if game.headers["White"] == "Carlsen, Magnus" else "black"
        
        for move in game.mainline_moves():
            if(color == "white" and board.turn == chess.WHITE) or (color == "black" and board.turn == chess.BLACK):
                data.append({
                    "board": fen_to_board(board.fen()).reshape(8, 8),
                    "color": color,
                    "move": move,
                })
            board.push(move)
        
df = pd.DataFrame(data)
df

,board,color,move
0,"[[r, n, b, q, k, b, n, r], [p, p, p, p, p, p, ...",white,e2e4
1,"[[r, n, b, q, k, b, n, r], [p, p, ., p, p, p, ...",white,g1f3
2,"[[r, n, b, q, k, b, n, r], [p, p, ., ., p, p, ...",white,d2d4
3,"[[r, n, b, q, k, b, n, r], [p, p, ., ., p, p, ...",white,f3d4
4,"[[r, n, b, q, k, b, ., r], [p, p, ., ., p, p, ...",white,d1e2
...,...,...,...
99,"[[., ., ., ., ., ., ., .], [., ., ., ., ., ., ...",white,g4g5
100,"[[., ., ., ., ., ., ., .], [., ., ., ., ., ., ...",white,g2g3
101,"[[., ., ., ., ., ., ., .], [., ., ., ., ., ., ...",white,g3f3
102,"[[., ., ., ., ., ., ., .], [., ., ., ., ., ., ...",white,f3g4


## 1.3 Visualize Data
Visualize the board to make sure we're chilling.


In [208]:
for i in range(5):
    print(df['board'][i])
    print(df['move'][i])
    print('\n')

[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['P' 'P' 'P' 'P' 'P' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]
e2e4


[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' '.' 'p' 'p' 'p' 'p' 'p']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' 'p' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'P' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['P' 'P' 'P' 'P' '.' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]
g1f3


[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' '.' '.' 'p' 'p' 'p' 'p']
 ['.' '.' '.' 'p' '.' '.' '.' '.']
 ['.' '.' 'p' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'P' '.' '.' '.']
 ['.' '.' '.' '.' '.' 'N' '.' '.']
 ['P' 'P' 'P' 'P' '.' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' '.' 'R']]
d2d4


[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' '.' '.' 'p' 'p' 'p' 'p']
 ['.' '.' '.' 'p' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '

## 1.4 Clean Data
Clean data.

In [209]:
df['color'] = df['color'].apply(lambda x: 1 if x == 'white' else 0)
# df['move'] = df['move'].astype('category')
df.dtypes

board    object
color     int64
move     object
dtype: object

## 1.5 Split Data

In [210]:
df_train = df.sample(frac=0.8)
df_valid = df.drop(df_train.index)

X_train = df_train.drop('move', axis=1)
X_valid = df_valid.drop('move', axis=1)
y_train = df_train['move']
y_valid = df_valid['move']

# 2 Train Model
## 2.1 Create Model

In [211]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,  # minimium amount of change to count as an improvement
    patience=20,  # how many epochs to wait before stopping
    restore_best_weights=True,
)

model = 

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
)

ValueError: All `outputs` values must be KerasTensors. Received: outputs=<Dense name=dense_52, built=False> including invalid value <Dense name=dense_52, built=False> of type <class 'keras.src.layers.core.dense.Dense'>

## 2.2 Fit Model

In [187]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=1,
    epochs=500,
    callbacks=[early_stopping],
    verbose=0,
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()

model.save('deeplearningmodel.h5')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).